In [1]:
from elasticsearch import Elasticsearch

from konlpy.tag import Kkma, Mecab

from datasets import load_from_disk

import time
import json
from contextlib import contextmanager

### 참고
https://amboulouma.com/elasticsearch-python

https://gh402.tistory.com/51

https://jvvp.tistory.com/1152

In [2]:
# Load wikipedia json data
with open('../../data/wikipedia_documents.json', 'r', encoding='utf-8') as f:
    wiki = json.load(f)

wiki_contents = list(dict.fromkeys([v['text'] for v in wiki.values()]))

In [3]:
# Check wikipedia data
print('Original data length:', len(wiki.keys()))
print('W/o replicated data length:', len(wiki_contents))

Original data length: 60613
W/o replicated data length: 56737


In [4]:
es = Elasticsearch('http://localhost:30001')

In [5]:
# Test Elasticsearch Connection
!curl -XGET localhost:30001

{
  "name" : "iCfG3OW",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "ne-ovptgRn-Gbd331afwZw",
  "version" : {
    "number" : "5.4.3",
    "build_hash" : "eed30a8",
    "build_date" : "2017-06-22T00:34:03.743Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.1"
  },
  "tagline" : "You Know, for Search"
}


In [6]:
# Check Cluster status
!curl -XGET localhost:30001/_cat/health?v

epoch      timestamp cluster       status node.total node.data shards pri relo init unassign pending_tasks max_task_wait_time active_shards_percent
1651647195 06:53:15  elasticsearch yellow          1         1     10  10    0    0       10             0                  -                 50.0%


In [7]:
# Get Index LIst
!curl -XGET localhost:30001/_cat/indices?v

health status index      uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   wiki_nouns skZkXThZTbaypb3JarTADg   5   1      56737            0     95.3mb         95.3mb
yellow open   wiki       ufIaLl2KQGKrA5iL6hGx8Q   5   1      56737            0    189.3mb        189.3mb


In [8]:
# Create Index (이미 해당 index 가 존재하면 에러남)
index = 'wiki_nouns'
if not es.indices.exists(index):
    es.indices.create(index=index)

In [23]:
# Get Index LIst  ->  생성한 index 가 보여야 함!
!curl -XGET localhost:30001/_cat/indices?v

health status index      uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   wiki_nouns skZkXThZTbaypb3JarTADg   5   1          0            0       650b           650b
yellow open   wiki       ufIaLl2KQGKrA5iL6hGx8Q   5   1      56737            0    189.3mb        189.3mb


In [16]:
# Index 삭제
# es.indices.delete(index='wiki')

{'acknowledged': True}

In [9]:
# Insert a data (Index should be existed!)
# Only Do Once!

kkma = Kkma()
mecab = Mecab()

# for idx, text in enumerate(wiki_contents):
#     # body = {'text': ' '.join(kkma.nouns(text))}
#     body = {'text': ' '.join(mecab.nouns(text))}
#     # body = {'text': wiki_contents[idx]}
#     es.index(index=index, doc_type="news", id=idx+1, body=body)
#     print(f'current: {idx}', end='\r')


In [10]:
# Get the data
es.get(index=index, doc_type='news', id=len(wiki_contents))

{'_index': 'wiki_nouns',
 '_type': 'news',
 '_id': '56737',
 '_version': 1,
 'found': True,
 '_source': {'text': '협약 부당 노동 행위 제도 규율 협약 조 반 노동조합 차별 행위 보호 규정 노동조합 가입 노동조합 탈퇴 것 조건 고용 황견계약 노동조합원 노동조합 활동 이유 이익 조치 것 보호 규정 조 노동자 단체 사용자 단체 사이 상호 간 간섭 보호 규정 사용 사용 단체 지배 하 둘 목적 노동자 단체 설립 지원 노동자 단체 재정 밖 방법 지원 것 간섭 행위 노동 조건 단체 협약 규율 사용 사용 단체 노동자 단체 사이 자발 교섭 기구 발전 이용 촉진 규정'}}

In [13]:
# Delete a data
# es.delete(index=index, id=1)

In [14]:
# Search the data by Query (Main !!!!)
# body = {
#     'query': {
#         'match': {
#             'text': '유엔 국제 기구'
#         }
#     }
# }

# res = es.search(index=index, body=body)

In [11]:
org_dataset = load_from_disk('../../data/train_dataset')

org_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [12]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")

In [13]:
# Inference for Train Validation

for i in range(1, 21):
    with timer(f'TOP K: {i}'):
        TOPK = i
        doc_scores = []
        doc_indices = []
        for j in range(len(org_dataset['validation'])):
            # By match
            body = {
                'size': TOPK,
                'query': {
                    'match': {
                        # 'text': ' '.join(kkma.nouns(org_dataset['validation']['question'][j]))
                        'text': ' '.join(mecab.nouns(org_dataset['validation']['question'][j]))
                    }
                }
            }

            
            # By match_phrase  =>  Fail
            # body = {
            #     'size': TOPK,
            #     'query': {
            #         'match_phrase': {
            #             # 'text': ' '.join(kkma.nouns(org_dataset['validation']['question'][j]))
            #             'text': ' '.join(mecab.nouns(org_dataset['validation']['question'][j]))
            #         }
            #     }
            # }


            # By terms  =>  not better than match
            # body = {
            #     'size': TOPK,
            #     'query': {
            #         'bool': {
            #             'should': [],
            #             'minimum_should_match': 3
            #         }
            #     }
            # }
            # for noun in mecab.nouns(org_dataset['validation']['question'][j]):
            #     body['query']['bool']['should'].append({'term': {'text': noun}})

            res = es.search(index=index, body=body)

            a_result_scores = []
            a_result_indices = []

            for item in res['hits']['hits']:
                a_result_scores.append(item['_score'])
                a_result_indices.append(int(item['_id'])-1)

            doc_scores.append(a_result_scores)
            doc_indices.append(a_result_indices)


        # Context Accuracy

        # correct = 0
        # for idx, doc_indice in enumerate(doc_indices):
        #     for jdx, indice in enumerate(doc_indice):
        #         if org_dataset['validation']['context'][idx] == wiki_contents[indice]:
        #             correct += 1

        # Label Accuracy  =>  more reasonable ?
        correct = 0
        for idx, doc_indice in enumerate(doc_indices):
            for jdx, indice in enumerate(doc_indice):
                if org_dataset['validation']['answers'][idx]['text'][0] in wiki_contents[indice]:
                    correct += 1
                    break
        
        print(f"Total Validation Score: {correct/len(org_dataset['validation'])*100}%")

Total Validation Score: 72.5%
[TOP K: 1] done in 1.195 s
Total Validation Score: 78.75%
[TOP K: 2] done in 1.365 s
Total Validation Score: 80.83333333333333%
[TOP K: 3] done in 1.472 s
Total Validation Score: 82.91666666666667%
[TOP K: 4] done in 1.610 s
Total Validation Score: 83.75%
[TOP K: 5] done in 1.711 s
Total Validation Score: 85.83333333333333%
[TOP K: 6] done in 1.759 s
Total Validation Score: 87.91666666666667%
[TOP K: 7] done in 1.883 s
Total Validation Score: 89.16666666666667%
[TOP K: 8] done in 1.947 s
Total Validation Score: 90.41666666666667%
[TOP K: 9] done in 2.038 s
Total Validation Score: 90.41666666666667%
[TOP K: 10] done in 2.170 s
Total Validation Score: 90.83333333333333%
[TOP K: 11] done in 2.182 s
Total Validation Score: 90.83333333333333%
[TOP K: 12] done in 2.221 s
Total Validation Score: 90.83333333333333%
[TOP K: 13] done in 2.258 s
Total Validation Score: 90.83333333333333%
[TOP K: 14] done in 2.307 s
Total Validation Score: 91.25%
[TOP K: 15] done in 2